In [ ]:
# this part of the codes fetches requirements
import hashlib
from phe import paillier
import random as rd
import numpy as np

In [ ]:
listOfIDS = []
dictionaryOfPublicKeys = {}
dictionaryOfPrivateKeys = {}
dictionaryOfRandomNumbers = {}
dictionaryOfBids = {}
finalSubmittedDataFirstPhase = [] #will be sused to stored the final resuklt of the two bidder for bid submission
highestbidder = 0 
numberOfBidders = 0 
publicKeyAuc, privateKeyAuc = 0, 0

In [ ]:
#  this part of the code creates unique BID for all participants
def createBiddersAndKeys(n):
  global numberOfBidders,dictionaryOfPublicKeys,dictionaryOfPrivateKeys,publicKeyAuc,privateKeyAuc
  numberOfBidders = n
  for i in range(0,numberOfBidders):
    i = str(i)
    listOfIDS.append(hashlib.sha256(i.encode('utf-8')).hexdigest())
  i = 0
  publicKeyAuc, privateKeyAuc = paillier.generate_paillier_keypair(n_length= 32)
  for i in range(0, numberOfBidders):
    x,y = paillier.generate_paillier_keypair(n_length=64)
    dictionaryOfPublicKeys[listOfIDS[i]],dictionaryOfPrivateKeys[listOfIDS[i]]  = x ,y
  i = 0

end of the Registration phase and start of Bid Submission Phase

In [ ]:
# this part of the code generate random number and distinct bid values for all bidders and Auctioneer
def  createRandomNumberAndBids():
    global dictionaryOfBids,dictionaryOfRandomNumbers
    bids  = rd.sample(range(1, numberOfBidders+1 ), numberOfBidders)
    for i in range(0,numberOfBidders):
        r  = rd.randint(100,10000)
        dictionaryOfRandomNumbers[listOfIDS[i]] =  r
        dictionaryOfBids[listOfIDS[i]] = bids[i]
    i = 0
    print(dictionaryOfBids)
    print(dictionaryOfRandomNumbers)


In [ ]:
# this PART OF THE CODE SIMULATES BIDDER GENERATING PRODUCT AND THAN SENDING TO NEXT BIDDER

def initFirstPhase():
    global finalSubmittedDataFirstPhasefor i in range(0,numberOfBidders):
    id = listOfIDS[i]
    dataPerBidder= []
    # print(dictionaryOfBids[id] * dictionaryOfRandomNumbers[id])
    encrytionOfBidAndRandomNumber = publicKeyAuc.raw_encrypt(dictionaryOfBids[id] * dictionaryOfRandomNumbers[id])
    # this part of code is forwarding the encrytionOfBidAndRandomNumber to all other bidders and after multiplication of the random number of the next
    # bidder it is sent to the Auctioneer immediately
    for j in range(0,numberOfBidders):
        if j != i:
            idOfNextBidder = listOfIDS[j]
            pubKeyNextBidder, priKeyNextBidder= dictionaryOfPublicKeys[idOfNextBidder],dictionaryOfPrivateKeys[idOfNextBidder]
            e = pubKeyNextBidder.raw_encrypt(encrytionOfBidAndRandomNumber)
            e = priKeyNextBidder.raw_decrypt(e)
            encrytionOfBidAndBothRandomNumber = e ** dictionaryOfRandomNumbers[idOfNextBidder]
            # this part of the code simulates the storing of the data of the bid summision to auctioneer data
            dataPerBidder.append(privateKeyAuc.raw_decrypt(encrytionOfBidAndBothRandomNumber))
        else:
            dataPerBidder.append(0)
    finalSubmittedDataFirstPhase.append(dataPerBidder)
    i,j = 0,0

In [ ]:
for i in range(len(finalSubmittedDataFirstPhase)):
    print(finalSubmittedDataFirstPhase[i])
i = 0

implementation of the second phase here auctioneer reforward each packet to every bidder to multiply their random number.

In [ ]:
#this function works as the intermediate steps where we are requesting each bidder to multiplyt his random number to the given ciphertext
productOfRandomNumbers = 0
for k in range(numberOfBidders):
    productOfRandomNumbers = productOfRandomNumbers + dictionaryOfRandomNumbers[listOfIDS[k]]
productOfRandomNumbers

In [ ]:
def everyBiddersRandomNumberMultiplication(x):
    x = np.power(x,productOfRandomNumbers)
    return x.item()

In [123]:
finalDataOfSecondPhase = []
auctioneerRandomNumber = rd.randint(2,2)
print(auctioneerRandomNumber)
for i in range(len(finalSubmittedDataFirstPhase)):
    dataPerBidder = []
    for j in range(numberOfBidders):
        if finalSubmittedDataFirstPhase[i][j] == 0 :
            dataPerBidder.append(0)
        else:
            encryptedValueByAuctioneer = publicKeyAuc.raw_encrypt(finalSubmittedDataFirstPhase[i][j] * auctioneerRandomNumber)
            returnValue = everyBiddersRandomNumberMultiplication(encryptedValueByAuctioneer)
            dataPerBidder.append((privateKeyAuc.raw_decrypt(returnValue)//auctioneerRandomNumber))
    finalDataOfSecondPhase.append(dataPerBidder)
i,j = 0,0


2


In [ ]:

for i in range(len(finalDataOfSecondPhase)):
    print(finalDataOfSecondPhase[i])
i = 0